In [ ]:
# !pip install azure-storage-blob
# !pip install python-dotenv

import os
from dotenv import load_dotenv
from setup_utils import fetch_data, load_data, create_time_index
import pandas as pd
import numpy as np

CONNECTION_STRING = os.getenv("CONNECTION_STRING")

load_dotenv()
fetch_data(CONNECTION_STRING)

(
    brand_mapping,
    macro_data,
    brand_constraint,
    pack_constraint,
    segment_constraint,
    sales_data,
    volume_variation_constraint,
) = load_data()

(
    macro_data,
    sales_data,
) = create_time_index([macro_data, sales_data])


In [ ]:
from sklearn.metrics import make_scorer, r2_score

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

In [ ]:
target_sku_list = sales_data[sales_data.gto.isna()].sku.unique()
target_brand_list = sales_data[sales_data.gto.isna()][["sku", "brand"]].brand.unique()

In [ ]:
k = sales_data[["sku", "brand"]].drop_duplicates().groupby("sku").apply(lambda x: set(x.brand.unique()))#[target_sku_list]
k2 = sales_data[["sku", "brand"]].drop_duplicates().groupby("sku").apply(lambda x: x.brand.nunique()>1)

issue_skus = k[k2].drop_duplicates().to_list()

brand_date_range = sales_data.fillna(0).reset_index().groupby("brand").apply(lambda x: pd.Series([x.date.min(), x.date.max()], index=["min", "max"]))
brand_date_range = brand_date_range.to_dict()

In [ ]:
# [(tuple(i)[0] in target_brand_list, tuple(i)[1] in target_brand_list) for i in issue_skus]
# sales_data[["sku", "brand"]].drop_duplicates().groupby("brand").apply(lambda x: x.sku.nunique())[list(set.union(*issue_skus))]
# [(brand_date_range["max"][tuple(i)[0]], brand_date_range["max"][tuple(i)[1]]) for i in issue_skus]
# [(brand_date_range["min"][tuple(i)[0]], brand_date_range["min"][tuple(i)[1]]) for i in issue_skus]

In [ ]:
# sales_data.reset_index().groupby(["date", "brand"]).volume.sum().unstack(1).T.loc[target_brand_list]

In [ ]:
# sales_data.reset_index().groupby(["date", "sku"]).volume.sum().unstack(1).T

In [ ]:
df = sales_data.reset_index().groupby(["date"])[["volume", "net_revenue", "promotional_discount", "other_discounts"]].sum().sort_index().join(macro_data)
k = df.net_revenue.shift(1)
k.name = "shifted_nr"
df = df.join(k).fillna(method="bfill")

In [ ]:
import tensorflow as tf

In [ ]:
data = df.iloc[:-2].copy(deep=True)

scaled_cols = ["net_revenue", "promotional_discount", "other_discounts", "shifted_nr"]#, "private_consumption", "gross_domestic_saving", "brnd_money", "gdp", "shifted_nr"]
scaler = data.net_revenue.mean()
data.loc[:,scaled_cols] = data.loc[:,scaled_cols]/scaler


mixed_effect_cols = ["retail_sales_index", "unemployment_rate", "cpi", "private_consumption", "gross_domestic_saving", "brad_money", "gdp"]

data.loc[:, mixed_effect_cols] = data.loc[:, mixed_effect_cols].divide(data.loc[:, mixed_effect_cols].mean())-1

In [ ]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()

In [ ]:
sess = tf.compat.v1.Session()

# Y
# y = tf.constant(data.net_revenue, dtype=tf.float64)
# discounts = -tf.constant(data[["promotional_discount", "other_discounts"]].values, dtype=tf.float64)
# mixed_effect = tf.constant(data[mixed_effect_cols].values, dtype=tf.float64)
# time_index = tf.constant(np.arange(1, data.shape[0]+1), dtype=tf.float64)
# shifted_nr = tf.constant(data.shifted_nr, dtype=tf.float64)

#Y
y = tf.compat.v1.placeholder(dtype=tf.float64, name="y_actual")

# X
shifted_nr = tf.compat.v1.placeholder(dtype=tf.float64, name="shifted_nr")
discounts = tf.compat.v1.placeholder(dtype=tf.float64, name="discounts")
mixed_effect = tf.compat.v1.placeholder(dtype=tf.float64, name="mixed_effects")
time_index = tf.compat.v1.placeholder(dtype=tf.float64, name="time_index")



# variables
baseline_intercept = tf.Variable(1, dtype=tf.float64)
baseline_slope1 = tf.Variable(1, dtype=tf.float64)
baseline_slope2 = tf.Variable(1, dtype=tf.float64)
mixed_effect_mult = tf.Variable(np.random.normal(loc=1, size=(1, 7)), dtype=tf.float64)
discount_slope = tf.math.sigmoid(tf.Variable(np.random.normal(loc=-1, size=(1, 2)), dtype=tf.float64))*3
roi_mults = tf.Variable(np.random.normal(loc=1, size=(1, 7)), dtype=tf.float64)

variable_list = [baseline_intercept, baseline_slope1, baseline_slope2, mixed_effect_mult, discount_slope]

# impacts
base1 = tf.multiply(baseline_slope1, time_index) + baseline_intercept
base2 = base1# + tf.multiply(baseline_slope2, shifted_nr)
mixed_effect_impact = 1 + tf.nn.tanh(tf.multiply(mixed_effect, mixed_effect_mult))
total_mixed_effect_impact = tf.reduce_prod(mixed_effect_impact, axis=1)
discount_impact = tf.multiply(discount_slope, discounts)
roi_mult_impact = 1 + tf.nn.tanh(tf.multiply(mixed_effect_impact, roi_mults))
total_roi_mult_impact = tf.expand_dims(tf.reduce_prod(roi_mult_impact, axis=1), axis=1)

# prediction
y_pred = (
    tf.multiply(base2, total_mixed_effect_impact)
    + tf.reduce_sum(discount_impact, axis=1)
)

# loss
wape = tf.reduce_sum(tf.math.abs(y - y_pred))/tf.reduce_sum(y)
mse = tf.reduce_sum(tf.math.square(y - y_pred))
reg = sum([tf.reduce_sum(tf.square(i)) for i in variable_list])

loss = 1e3*wape + 1e2*mse + reg

In [ ]:
feed_dict1 = {
    discounts : -data[["promotional_discount", "other_discounts"]].iloc[:-5,:].astype(np.float64).values,
    mixed_effect: data[mixed_effect_cols].iloc[:-5,:].astype(np.float64).values,
    shifted_nr : data["shifted_nr"].iloc[:-5].astype(np.float64).values,
    y : data["net_revenue"].iloc[:-5].astype(np.float64).values,
    time_index : np.float64(np.arange(1, data.iloc[:-5,:].shape[0]+1)),
}

feed_dict2 = {
    discounts : -data[["promotional_discount", "other_discounts"]].iloc[-5:,:].astype(np.float64).values,
    mixed_effect: data[mixed_effect_cols].iloc[-5:,:].astype(np.float64).values,
    shifted_nr : data["shifted_nr"].iloc[-5:].astype(np.float64).values,
    y : data["net_revenue"].iloc[-5:].astype(np.float64).values,
    time_index : np.float64(np.arange(1, data.iloc[-5:,:].shape[0]+1)),
}

In [ ]:
epoch = 0
# optimizer
lr = lambda x : 0.1 / np.power(x/100 + 10, 2 / 3)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=lr(epoch))#, beta1=0.1, beta2=0.1)
train = optimizer.minimize(loss)

# initialize variables
init = tf.compat.v1.global_variables_initializer()
sess.run(init, feed_dict1)

In [ ]:
# train model
num_epochs = 30000
for epoch in range(num_epochs):
    _, current_loss, current_wape, current_mse, current_reg = sess.run([train, loss, wape, mse, reg], feed_dict1)
    if (epoch + 1) % 250 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {current_loss:.4f}, WAPE: {current_wape:.4f}, MSE: {current_mse:.4f}, reg: {current_reg:.4f}")


#         # Training loop
# num_epochs = 500
# for epoch in range(num_epochs):
#     _, current_error, cuurent_mse, current_m1, current_m2, current_c = sess.run([train_op, error, mse_error, m1, m2, c])
#     if (epoch + 1) % 25 == 0:
#         print(f"Epoch {epoch + 1}/{num_epochs}, Error: {current_error:.4f}, MSE: {cuurent_mse:.4f}, m1: {current_m1}, m2: {current_m2}, c: {current_c}")

# # Print the final results for 'm' and 'c'
# final_m1, final_m2, final_c = sess.run([m1, m2, c])
# print(f"Final 'm1' value: {final_m1}")

# print(f"Final 'm2' value: {final_m2}")
# print(f"Final 'c' value: {final_c}")

In [ ]:
# train model
num_epochs = 30000
for epoch in range(num_epochs):
    _, current_loss, current_wape, current_mse, current_reg = sess.run([train, loss, wape, mse, reg], feed_dict1)
    if (epoch + 1) % 250 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {current_loss:.4f}, WAPE: {current_wape:.4f}, MSE: {current_mse:.4f}, reg: {current_reg:.4f}")


#         # Training loop
# num_epochs = 500
# for epoch in range(num_epochs):
#     _, current_error, cuurent_mse, current_m1, current_m2, current_c = sess.run([train_op, error, mse_error, m1, m2, c])
#     if (epoch + 1) % 25 == 0:
#         print(f"Epoch {epoch + 1}/{num_epochs}, Error: {current_error:.4f}, MSE: {cuurent_mse:.4f}, m1: {current_m1}, m2: {current_m2}, c: {current_c}")

# # Print the final results for 'm' and 'c'
# final_m1, final_m2, final_c = sess.run([m1, m2, c])
# print(f"Final 'm1' value: {final_m1}")

# print(f"Final 'm2' value: {final_m2}")
# print(f"Final 'c' value: {final_c}")

In [ ]:
np.absolute(sess.run(y, feed_dict1)-sess.run(y_pred, feed_dict1)).sum()/sess.run(y, feed_dict1).sum()

In [ ]:
np.absolute(sess.run(y, feed_dict2)-sess.run(y_pred, feed_dict2)).sum()/sess.run(y, feed_dict2).sum()

In [ ]:
r2_score(sess.run(y, feed_dict1), sess.run(y_pred, feed_dict1))

In [ ]:
pd.DataFrame({"y":sess.run(y, feed_dict1), "y_pred" : sess.run(y_pred, feed_dict1)}).plot()

In [ ]:
sess.run(variable_list, feed_dict)

In [ ]:
# pd.DataFrame(sess.run([base1, base2-base1, tf.multiply(base2, mixed_effect_impact)-base2, tf.reduce_sum(discount_impact, axis=1)])).T.plot()

In [ ]:
# # Import the necessary libraries
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.preprocessing import MinMaxScaler

# # Generate or load your dataset
# X, y = df.iloc[:-2, 2:], df.iloc[:-2, 1]

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a MinMaxScaler and fit it to the training data
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Create a linear regression model
# model = LinearRegression()

# # Fit the model to the scaled training data
# model.fit(X_train_scaled, y_train)

# # Make predictions on the scaled test data
# y_pred = model.predict(X_test_scaled)

# # Calculate and print performance metrics
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Mean Squared Error:", mse)
# print("R-squared:", r2)

# # Optionally, you can also access the model's coefficients and intercept
# coefficients = model.coef_
# intercept = model.intercept_

# print("Coefficients:", coefficients)
# print("Intercept:", intercept)
